In [1]:
import pandas as pd

In [2]:
tbl = pd.read_csv("test.smbtbl", delimiter='|')
tbl.set_index("MLo", inplace=True)
print(tbl.columns)
tbl

Index(['Type', 'Lenght', 'Name', 'Scope', 'LineNumbers'], dtype='object')


,Type,Lenght,Name,Scope,LineNumbers
MLo,,,,,
0,int,0,n,main,40 41 42 43 43 45
2,void,0,output,global,45
1,int,0,input,global,41
-1,void,0,main,global,38


In [3]:
file_path = 'test.quad'
commands = []
with open(file_path, 'r') as file:
    for line in file:
        quad = line.strip('(')
        quads = quad[:-2].split(', ')
        commands.append(quads)
print(commands[0][3])

-


In [4]:
# Fix for Call registers ($t28 is return reg)
for i, command in enumerate(commands):
    if command[0] == "CALL":
        aux = command[1]
        command[1] = "$t28"
        for comando in commands[i:]:
            for j in range(1,4):
                if comando[j] == aux:
                    comando[j] = "$t28"
for command in commands:
    print(command)

['FUNC', 'void', 'main', '-']
['ALLOC', 'n', 'main', '-']
['CALL', '$t28', 'input', '0']
['ADD', '$t1', '$t0', '$t28']
['STORE', 'n', '$t1', '-']
['LAB', 'L0', '-', '-']
['LOAD', '$t2', 'n', '-']
['ADDI', '$t3', '$t0', '3']
['LT', '$t4', '$t2', '$t3']
['WHILE', '$t4', 'L1', '-']
['LOAD', '$t5', 'n', '-']
['ADDI', '$t6', '$t0', '1']
['SUB', '$t7', '$t5', '$t6']
['STORE', 'n', '$t7', '-']
['GOTO', 'L0', '-', '-']
['LAB', 'L1', '-', '-']
['LOAD', '$t8', 'n', '-']
['PARAM', '$t8', '-', '-']
['CALL', '$t28', 'output', '1']
['ADD', '$t9', '$t0', '$t28']
['END', 'main', '-', '-']


In [5]:

usedIndexes = []
def fixIndex(line):
    global usedIndexes
    tobeUsed = []
    for command in commands[line + 1:]:
        for i in range(1,len(command)):
            if "$t" in command[i]:
                if command[i][2:] in usedIndexes and command[i][2:] not in tobeUsed:
                    tobeUsed.append(command[i][2:])
    usedIndexes = list(set(usedIndexes) - set(set(usedIndexes) - set(tobeUsed)))

In [6]:
def getIndex(reg, indice, indice2):
    if "$t" in commands[indice][indice2]:
        index = int(reg[2:])
        if index == 0:
            return commands[indice][indice2]
        if index < 29:
            return commands[indice][indice2]
        for num in range(1, 28):
            if str(num) not in usedIndexes:
                for i in range(0,len(commands)):
                    for j in range(1,len(commands[i])):
                        if "$t" in commands[i][j]:
                            if commands[i][j] == reg:
                                commands[i][j] = "$t"+str(num)
                                break
                usedIndexes.remove(str(index))
                usedIndexes.append(str(num))
                return "$t"+str(num)
        print("\n----------------------------\nFATAL ERROR: NOT ENOUGH REGISTERS!!!\n----------------------------\n")
    return "FATAL ERROR"

In [7]:
#addi beq bne bgt blt lw sw j jal (jar) add sub div mult and or
usedVars = {}
Params = []
assembly = open("test.asm", 'w')
scope = " "
Callflag = 0
assembly.write("j "+"main""\n")
for i, command in enumerate(commands):
    for reg in command[1:]:
        if "$t" in reg and reg[2:] not in usedIndexes:
            usedIndexes.append(reg[2:])
    match command[0]:
        case 'FUNC':
            assembly.write("LAB "+command[2]+"\n")
            scope = command[2]
            if command[2] != "main":
                assembly.write("addi $t30 $t30 1\n")
                assembly.write("sw $t31 $t29 1"+"\n")
        case 'END':
            if command[1] != 'main':
                assembly.write("LAB FIM:"+command[1] +"\n")
                assembly.write("lw $t31 $t29 1"+"\n")
                assembly.write("jr $t31"+"\n")
            else:
                assembly.write("halt"+"\n")

        case 'ARG':
            assembly.write("addi $t30 $t30 1\n")
        case 'LOAD':
            reg = getIndex(command[1],i,1)
            if command[2] not in usedVars:
                usedVars[command[2]] = reg
            assembly.write("lw "+reg+" "+"$t29 "+str(tbl[(tbl['Name'] == command[2]) & (tbl['Scope'] == scope)].index[0])+"\n")
        case 'GOTO':
            assembly.write("j "+command[1]+"\n")
        case 'LAB':
            assembly.write("LAB "+command[1]+"\n")
        case 'IFF':
            assembly.write("beq "+command[1]+" "+"$t0 "+command[2]+"\n")
        case 'WHILE':
            assembly.write("beq "+command[1]+" "+"$t0 "+command[2]+"\n")
        case 'RET':
            if command[1] == "$t28":
                assembly.write("addi $t28 "+ "$t28 "+"0"+"\n")
            else:
                assembly.write("addi $t28 "+ getIndex(command[1],i,1)+" 0"+"\n")
            assembly.write("j FIM:"+scope+"\n")
        case 'CALL':
            if command[2] == "input":
                assembly.write("input " +"$t28"+"\n")
            elif command[2] == "output":
                assembly.write("output " +Params[0]+"\n")
            else:
                backup = usedVars.copy()
                for var in list(usedVars):
                    assembly.write("sw "+usedVars[var]+" "+"$t30 "+ "0"+"\n")
                    assembly.write("addi " +"$t30 " +"$t30 "+ "1"+"\n")
                    del usedVars[var]
                assembly.write("sw " +"$t29 " +"$t30 "+ "0"+"\n")
                assembly.write("addi " +"$t29 " +"$t30 "+ "0"+"\n")
                assembly.write("addi " +"$t30 " +"$t30 "+ "1"+"\n")
                Mem = 2
                for param in Params:
                    assembly.write("sw "+param+" $t29 "+str(Mem)+"\n")
                    Mem += 1
                Mem = 2
                Params = []
                assembly.write("jal " +command[2]+"\n")
                assembly.write("addi " +"$t30 " +"$t29 "+ "0"+"\n")
                assembly.write("lw " +"$t29 " +"$t29 "+ "0"+"\n")
                usedVars = backup
                for var in list(usedVars):
                    assembly.write("subi " +"$t30 " +"$t30 "+ "1"+"\n")
                    assembly.write("lw "+usedVars[var]+" "+"$t30 "+ "0"+"\n")
                    del usedVars[var]
        case 'ADDI':
            #command[2] is always $t0
            assembly.write("addi "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+command[3]+"\n")
        case 'ALLOC':
            assembly.write("addi $t30 $t30 1\n")
        case 'STORE':
            assembly.write("sw "+getIndex(command[2],i,2) +" $t29 "+str(tbl[(tbl['Name'] == command[1]) & (tbl['Scope'] == scope)].index[0])+"\n")
        case 'PARAM':
            Params.append(getIndex(command[1],i,1))
        case 'ADD':
            assembly.write("add "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'SUB':
            assembly.write("sub "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'MULT':
            assembly.write("mult "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'DIV':
            assembly.write("div "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'LT':
            assembly.write("lt "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'LET':
            assembly.write("let "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'GT':
            assembly.write("gt "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'GET':
            assembly.write("get "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'COMP':# ==
            assembly.write("comp "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'NEQ':
            assembly.write("neq "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
    fixIndex(i)
assembly.close()

['STORE', 'n', '$t3', '-']


ValueError: list.remove(x): x not in list

In [ ]:
for command in commands:
    print(command)

In [ ]:
file_path = 'test.asm'
commands_asm = []
linha = 0
with open(file_path, 'r') as file:
    for line in file:
        quads = line.split(' ')
        commands_asm.append(quads)
        print(quads)

In [ ]:
file_path = "test.asmfinal"
asmfinal = open(file_path, 'w')
linha = 0
for command in commands_asm:
    if command[0] == "LAB":
        for comando in commands_asm:
            for j in range(0,len(comando)):
                if command[1] == comando[j] and command != comando:
                    comando[j] = str(linha) + "\n"
                    break
    else:
        linha +=1
    print(command)
print(linha)

In [ ]:
for command in commands_asm:
    print(command)

In [ ]:
linha = 0
for command in commands_asm:
    if command[0] != "LAB":
        linha += 1
        if command[0] == "beq":
            command[3] = str((int(command[3]) - linha)) + "\n"
        asmfinal.write(" ".join(command))
    print(command)
print(linha)
asmfinal.close()
